## **1. Introduction & Context**

This micro-project demonstrates basic SQL skills for data analysis within the domain of **capital adequacy** forecasting under Basel III regulations.

$$
\text{Capital Adequacy}\quad\text{=}\frac{TotalCapital}{RiskWeightedAssets}
$$

Capital Adequacy is our ratio of certain types of capital to our Risk-Weighted Assets, more detail on this will be explained in *Section 3. Data Modeling & Insertion*.

Basel III regulations are minimum capital, leverage, and liquidity requirements created by the Basel Committee on Banking Supervision, a consortium of central banks from 28 countries based in Basel, Switzerland, shortly after the financial crisis of 2007–2008.

We will create tables for fictional business units with their own balance sheets, then project adequacy ratios at different level of economic distress.

## **2. Schema Overview**

### 🗂️ Table Overview: Financial Stress Testing Scenario

Below is a brief description of each table used in this project, explaining its role in our simplified stress testing model.

---

#### 🏢 `business_units`

**Purpose**:  
This table defines each individual business unit within the organization.  
Each unit has a unique `unit_id` and a descriptive `unit_name`.

**Why it matters**:  
It provides a master list of business units that other tables reference.  
Serves as the anchor for linking financial and regulatory data to specific units.

---

#### 📊 `balance_sheets`

**Purpose**:  
Stores the historical (or baseline) balance sheet data for each business unit.  
Includes total `Common Equity Tier 1`, `Additional Tier 1`, `Tier 2` and `Risk-Weighted Assets` for a given `report_date`. More details on these will be provided in Section 3.

**Why it matters**:  
This is the core financial data used to evaluate how business units would perform under economic stress scenarios.  
It forms the basis for projections.

---

#### 🛡️ `regulatory_requirements`

**Purpose**:  
Contains the regulatory minimum capital ratio required for each capital tier.  
Each tier has a `min_capital_ratio` associated with it.

**Why it matters**:  
Provides a benchmark to compare against projected capital ratios.  
Used to determine whether a business unit would remain compliant under stress.

---

#### 📈 `forecast_assumptions`

**Purpose**:  
Holds the stress testing assumptions for different economic scenarios.  
Each row contains a `scenario_name` along with `cet1_multiplier`, `at1_multiplier`, `tier2_multiplier`, and `rwa_multiplier`.

**Why it matters**:  
Enables simulation of how financials might change under adverse or severely adverse conditions.  
These multipliers are applied to the baseline balance sheet data.

---

Since we are in Jupyter Notebooks I don't want to try to create redundant tables in our db file. Working in Jupyter is usually done when you want to break code into pieces and run multiple lines over and over as you iterate. So, we will drop the tables at the beginning of our CREATE TABLE statements. Our output should be 8 Done statements, one for each drop, and one for each create.

In [250]:
%reload_ext sql
%sql sqlite:///test_capital_project.db

'Connected: @test_capital_project.db'

In [251]:
%%sql

DROP TABLE IF EXISTS balance_sheets;
DROP TABLE IF EXISTS business_units;
DROP TABLE IF EXISTS regulatory_requirements;
DROP TABLE IF EXISTS forecast_assumptions;

CREATE TABLE business_units (
    unit_id INTEGER PRIMARY KEY,
    unit_name TEXT NOT NULL
);

CREATE TABLE balance_sheets (
    bank_id INTEGER,
    unit_id INTEGER,
    report_date,
    risk_weighted_exposure REAL,
    cet1_capital REAL,
    at1_capital REAL,
    tier2_capital REAL,
    FOREIGN KEY (unit_id) REFERENCES business_units(unit_id)
);

CREATE TABLE regulatory_requirements (
    buffer_id INTEGER PRIMARY KEY,
    tier TEXT,
    min_capital_ratio FLOAT NOT NULL
);

CREATE TABLE forecast_assumptions (
    assumption_id INTEGER PRIMARY KEY,
    scenario_name TEXT NOT NULL,
    cet1_multiplier FLOAT NOT NULL,
    at1_multiplier FLOAT NOT NULL,
    tier2_multiplier FLOAT NOT NULL,
    rwa_multiplier FLOAT NOT NULL
);

 * sqlite:///test_capital_project.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## **3. Data Modeling & Insertion**

We will populate each of our 4 tables with small, realistic sample data. First, our imaginary Business Units.

---

In [272]:
# Insert data into our business_units table

In [253]:
%%sql
INSERT INTO business_units (unit_id, unit_name) VALUES
(1, 'Retail Banking'),
(2, 'Commercial Lending'),
(3, 'Credit Cards'),
(4, 'Wealth Management');
SELECT * FROM business_units

 * sqlite:///test_capital_project.db
4 rows affected.
Done.


unit_id,unit_name
1,Retail Banking
2,Commercial Lending
3,Credit Cards
4,Wealth Management


---
# Capital Tiers and Risk Weighted Assets



The purpose of Basel III regulation and our forecasting is to test whether a bank has money on hand to cover losses in the case of emergencies. The calculation looks like this.

$$
\frac{*TotalCapital}{RiskWeightedAssets}\quad\text{must be greater than or equal to} \quad8\%
$$

**Risk Weighted Assets** - The value of assets on hand adjusted for risk. For example, a loan to someone with good credit would be valued differently to someone with bad credit, even if the loan terms were the same.


**Capital** - Assets left on hand after all obligations are paid. In other words, the cash we would have if the company went out of business.



All Capital is not valued the same, and are separated into Tiers, depending on how highly it is valued under Basel III regulations. The Tiers are:

1. Common Equity Tier 1 (CET1)
2. Additional Tier 1 (AT1)
3. Tier 2 (Tier2)

$$
\begin{array}{c}
  CET1 \\
+ AT1 \\
\hline
  Tier 1
\end{array}
$$


$$
\begin{array}{c}
  Tier1 \\
+ Tier2 \\
\hline
  *Total Capital
\end{array}
$$

We will insert Risk-Weighted Assets and Capital Tier values for two ending periods: one End-of-Year and one Mid-Year. When we created our tables, we assigned our unit_id columns as foreign keys to match to the primary key of the business_units table.

In [273]:
# Insert data into our balance_sheets table

In [254]:
%%sql

INSERT INTO balance_sheets (bank_id, unit_id, report_date, risk_weighted_exposure, cet1_capital, at1_capital, tier2_capital) VALUES
(1, 1, '2023-12-31', 4000000, 200000, 80000, 120000),
(2, 2, '2023-12-31', 7000000, 290000, 80000, 50000),
(3, 3, '2023-12-31', 5500000, 275000, 110000, 165000),
(4, 4, '2023-12-31', 7500000, 412500, 172500, 225000),
(5, 1, '2024-06-30', 4100000, 209100, 102500, 133400),
(6, 2, '2024-06-30', 7100000, 390500, 170500, 200000),
(7, 3, '2024-06-30', 5600000, 180000, 90000, 100000),
(8, 4, '2024-06-30', 8000000, 440000, 180000, 240000);

SELECT * FROM balance_sheets

 * sqlite:///test_capital_project.db
8 rows affected.
Done.


bank_id,unit_id,report_date,risk_weighted_exposure,cet1_capital,at1_capital,tier2_capital
1,1,2023-12-31,4000000.0,200000.0,80000.0,120000.0
2,2,2023-12-31,7000000.0,290000.0,80000.0,50000.0
3,3,2023-12-31,5500000.0,275000.0,110000.0,165000.0
4,4,2023-12-31,7500000.0,412500.0,172500.0,225000.0
5,1,2024-06-30,4100000.0,209100.0,102500.0,133400.0
6,2,2024-06-30,7100000.0,390500.0,170500.0,200000.0
7,3,2024-06-30,5600000.0,180000.0,90000.0,100000.0
8,4,2024-06-30,8000000.0,440000.0,180000.0,240000.0


---

In the real world our calculations are not so simple as taking Total Capital, dividing it by Risk Weighted Assets, and being finished. Basel III also outlines requirements for how much of each Tier must make up the percentage value of RWA.

CET1/RWA must be greater than or equal to 8%. If it is not, it must be at least 4.5% **and**

Tier 1 (CET1 + AT1)/RWA must be greater than or equal to 8%. If neither are greater than or equal to 8%, then they must make up at least 6% **and**

Total Capital (CET1 + AT1 + Tier2)/RWA must be greater than or equal to 8%.

Our tables will dispense with some of the nuance of the requirements and just assume that we are not complaint if we don't meet the 4.5%, 6%, and 8% requirements. In reality, it's perfectly possible to make up the entire requirement with a compliant percentage of CET1 and AT1, or even CET1 entirely.

In [256]:
# Insert data into our regulatory_requirements table

In [257]:
%%sql
INSERT INTO regulatory_requirements (buffer_id, tier, min_capital_ratio) VALUES
(1, 'CET1', 0.045),
(2, 'Tier 1',0.06),
(3, 'Total', 0.08);
SELECT * FROM regulatory_requirements

 * sqlite:///test_capital_project.db
3 rows affected.
Done.


buffer_id,tier,min_capital_ratio
1,CET1,0.045
2,Tier 1,0.06
3,Total,0.08


---

Finally, we will insert data into our forecasting table. We will have four different economic scenarios with multiplicative effects on our capital and assets for each.

Our baseline will be the normal data with no changes.

A Mild Recession with see a shrinking of our capital and an increase in our RWA, represented by our riskier assets being weighted more heavily as everyone experiences economic distress,

Severe Recession has a more severe decrease in capital, but the `rwa_multiplier` is not as high to represent the banks divestiture of riskier assets during a bear market.

Finally, Expansion sees an increase in all of our fields, we have more equity to work with in the good times and are also confident we can absorb the loses from risker assets on our books.

In [258]:
# Insert data into our forecast_assumptions table

In [259]:
%%sql
INSERT INTO forecast_assumptions (assumption_id, scenario_name, cet1_multiplier, at1_multiplier, tier2_multiplier, rwa_multiplier) VALUES
(1, 'Baseline', 1.00, 1.00, 1.00, 1.00),
(2, 'Mild Recession', 0.98, .097, 0.95, 1.05),
(3, 'Severe Recession', 0.90, 0.85, 0.80, 1.20),
(4, 'Expansion', 1.03, 1.02, 1.01, 1.05);
SELECT * FROM forecast_assumptions

 * sqlite:///test_capital_project.db
4 rows affected.
Done.


assumption_id,scenario_name,cet1_multiplier,at1_multiplier,tier2_multiplier,rwa_multiplier
1,Baseline,1.0,1.0,1.0,1.0
2,Mild Recession,0.98,0.097,0.95,1.05
3,Severe Recession,0.9,0.85,0.8,1.2
4,Expansion,1.03,1.02,1.01,1.05


---

## **4. Forecasting Scenarios**

Now we have our tables for each of our 4 business units for the end of year and mid-year data, as well as our minimum capital ratios and scenarios.

Next, we will get started on forecasting our data, starting with an INNER JOIN our balance_sheets and business_units tables. We want to put our relevant starting information into a convenient table to work with and create a View.

A view is a virtual table based on the result of a query. In this case we want to run our calculation on our tier and RWA numbers for each unit and report date, so we will create a view to more conveniently work with the data in our next query and name it total_capital.

In [274]:
# Create view with total_capital

In [261]:
%%sql
DROP VIEW IF EXISTS total_capital;

CREATE VIEW total_capital AS
SELECT 
    bu.unit_name,
    bs.report_date,
    bs.cet1_capital,
    bs.at1_capital,
    bs.tier2_capital,
    bs.risk_weighted_exposure,
    ROUND(bs.cet1_capital + bs.at1_capital + bs.tier2_capital, 2) AS total_capital
FROM balance_sheets bs
INNER JOIN business_units bu
    ON bs.unit_id = bu.unit_id;
SELECT * FROM total_capital

 * sqlite:///test_capital_project.db
Done.
Done.
Done.


unit_name,report_date,cet1_capital,at1_capital,tier2_capital,risk_weighted_exposure,total_capital
Retail Banking,2023-12-31,200000.0,80000.0,120000.0,4000000.0,400000.0
Commercial Lending,2023-12-31,290000.0,80000.0,50000.0,7000000.0,420000.0
Credit Cards,2023-12-31,275000.0,110000.0,165000.0,5500000.0,550000.0
Wealth Management,2023-12-31,412500.0,172500.0,225000.0,7500000.0,810000.0
Retail Banking,2024-06-30,209100.0,102500.0,133400.0,4100000.0,445000.0
Commercial Lending,2024-06-30,390500.0,170500.0,200000.0,7100000.0,761000.0
Credit Cards,2024-06-30,180000.0,90000.0,100000.0,5600000.0,370000.0
Wealth Management,2024-06-30,440000.0,180000.0,240000.0,8000000.0,860000.0


To review, our regulatory requirements are:

1. cet1_ratio must be 4.5% or higher
2. tier1_ratio must be 6% or higher
3. total_ratio must be 8% or higher

We will use a CASE statement to ask if the requirements are met. If so, we will Pass. If not, Fail.

We are using our View with a Common Table Expression (CTE). A CTE is a temporary, named table defined with the WITH clause and can help create more readable queries. The query after this one could have used an additional CTE to name a calculated column, making the final SELECT statement more readable. It also would have been more convenient to create another view as an intermediary step. The contrast between this query and the next show how useful CTEs are to make queries readable.

Using our dummy data and the logic outlined above, we will see that two of our records fail, the rest pass.

In [275]:
# Showing pass/fail for each tier category

In [263]:
%%sql

WITH capital_data AS (
    SELECT
        unit_name,
        report_date,
        cet1_capital,
        at1_capital,
        tier2_capital,
        risk_weighted_exposure,
        (cet1_capital + at1_capital) AS tier1_capital,
        (cet1_capital + at1_capital + tier2_capital) AS total_capital
    FROM total_capital
),

ratios AS (
    SELECT
        unit_name,
        report_date,
        cet1_capital,
        at1_capital,
        tier2_capital,
        risk_weighted_exposure,
        tier1_capital,
        total_capital,
        ROUND(cet1_capital / risk_weighted_exposure, 4) AS cet1_ratio,
        ROUND(tier1_capital / risk_weighted_exposure, 4) AS tier1_ratio,
        ROUND(total_capital / risk_weighted_exposure, 4) AS total_ratio
    FROM capital_data
),

reg_requirements AS (
    SELECT 
        MAX(CASE WHEN tier = 'CET1' THEN min_capital_ratio END) AS min_cet1,
        MAX(CASE WHEN tier = 'Tier 1' THEN min_capital_ratio END) AS min_tier1,
        MAX(CASE WHEN tier = 'Total' THEN min_capital_ratio END) AS min_total
    FROM regulatory_requirements
)

SELECT
    r.unit_name,
    r.report_date,
    r.cet1_ratio,
    CASE WHEN r.cet1_ratio >= req.min_cet1 THEN 'PASS' ELSE 'FAIL' END AS cet1_pass,
    r.tier1_ratio,
    CASE WHEN r.tier1_ratio >= req.min_tier1 THEN 'PASS' ELSE 'FAIL' END AS tier1_pass,
    r.total_ratio,
    CASE WHEN r.total_ratio >= req.min_total THEN 'PASS' ELSE 'FAIL' END AS total_pass
FROM ratios r
CROSS JOIN reg_requirements req;

 * sqlite:///test_capital_project.db
Done.


unit_name,report_date,cet1_ratio,cet1_pass,tier1_ratio,tier1_pass,total_ratio,total_pass
Retail Banking,2023-12-31,0.05,PASS,0.07,PASS,0.1,PASS
Commercial Lending,2023-12-31,0.0414,FAIL,0.0529,FAIL,0.06,FAIL
Credit Cards,2023-12-31,0.05,PASS,0.07,PASS,0.1,PASS
Wealth Management,2023-12-31,0.055,PASS,0.078,PASS,0.108,PASS
Retail Banking,2024-06-30,0.051,PASS,0.076,PASS,0.1085,PASS
Commercial Lending,2024-06-30,0.055,PASS,0.079,PASS,0.1072,PASS
Credit Cards,2024-06-30,0.0321,FAIL,0.0482,FAIL,0.0661,FAIL
Wealth Management,2024-06-30,0.055,PASS,0.0775,PASS,0.1075,PASS


---

Now we will put everything together. We will run out pass/fail tests for each unit, reporting period, and scenario.

In [264]:
# Pass/Fail Test with forecasting

In [265]:
%%sql

WITH scenario_forecasts AS (
    SELECT 
        fa.scenario_name,
        bs.bank_id,
        bs.unit_id,
        bs.report_date,
        ROUND(bs.cet1_capital * fa.cet1_multiplier, 4) AS forecast_cet1,
        ROUND(bs.at1_capital * fa.at1_multiplier, 4) AS forecast_at1,
        ROUND(bs.tier2_capital * fa.tier2_multiplier, 4) AS forecast_tier2,
        ROUND(bs.risk_weighted_exposure * fa.rwa_multiplier, 4) AS forecast_rwa
    FROM balance_sheets bs
    CROSS JOIN forecast_assumptions fa
),

reg_requirements AS (
    SELECT 
        MAX(CASE WHEN tier = 'CET1' THEN min_capital_ratio END) AS min_cet1,
        MAX(CASE WHEN tier = 'Tier 1' THEN min_capital_ratio END) AS min_tier1,
        MAX(CASE WHEN tier = 'Total' THEN min_capital_ratio END) AS min_total
    FROM regulatory_requirements
)

SELECT 
    bu.unit_name,
    sf.report_date,
    sf.scenario_name,
    ROUND(sf.forecast_cet1 / sf.forecast_rwa,4) AS forecast_cet1_ratio,
    CASE WHEN ROUND(sf.forecast_cet1 / sf.forecast_rwa,4) >= ROUND((req.min_cet1 * sf.forecast_rwa)/sf.forecast_rwa,4) THEN 'PASS' ELSE 'FAIL' END AS cet1_pass,
    ROUND((sf.forecast_cet1 + sf.forecast_at1)/sf.forecast_rwa,4) AS forecast_tier1_ratio,
    CASE WHEN ROUND((sf.forecast_cet1 + sf.forecast_at1)/sf.forecast_rwa,4) >= ROUND((req.min_tier1 * sf.forecast_rwa/sf.forecast_rwa),4) THEN 'PASS' ELSE 'FAIL' END AS tier1_pass,
    ROUND((sf.forecast_cet1 + sf.forecast_at1 + sf.forecast_tier2)/sf.forecast_rwa,4) AS forecast_total_ratio,    
    CASE WHEN ROUND((sf.forecast_cet1 + sf.forecast_at1 + sf.forecast_tier2)/sf.forecast_rwa,4) >= ROUND((req.min_total * sf.forecast_rwa)/sf.forecast_rwa,4) THEN 'PASS' ELSE 'FAIL' END AS total_pass
FROM scenario_forecasts sf
JOIN business_units bu ON sf.unit_id = bu.unit_id
CROSS JOIN reg_requirements req
ORDER BY unit_name, report_date, scenario_name;

 * sqlite:///test_capital_project.db
Done.


unit_name,report_date,scenario_name,forecast_cet1_ratio,cet1_pass,forecast_tier1_ratio,tier1_pass,forecast_total_ratio,total_pass
Commercial Lending,2023-12-31,Baseline,0.0414,FAIL,0.0529,FAIL,0.06,FAIL
Commercial Lending,2023-12-31,Expansion,0.0406,FAIL,0.0517,FAIL,0.0586,FAIL
Commercial Lending,2023-12-31,Mild Recession,0.0387,FAIL,0.0397,FAIL,0.0462,FAIL
Commercial Lending,2023-12-31,Severe Recession,0.0311,FAIL,0.0392,FAIL,0.0439,FAIL
Commercial Lending,2024-06-30,Baseline,0.055,PASS,0.079,PASS,0.1072,PASS
Commercial Lending,2024-06-30,Expansion,0.054,PASS,0.0773,PASS,0.1044,PASS
Commercial Lending,2024-06-30,Mild Recession,0.0513,PASS,0.0536,FAIL,0.079,FAIL
Commercial Lending,2024-06-30,Severe Recession,0.0413,FAIL,0.0583,FAIL,0.077,FAIL
Credit Cards,2023-12-31,Baseline,0.05,PASS,0.07,PASS,0.1,PASS
Credit Cards,2023-12-31,Expansion,0.049,PASS,0.0685,PASS,0.0973,PASS


---

## **5. Takeaways**

While none of our departments meet our Target Ratio in the case of a Severe Recession, Retail Banking and Wealth Management remain resilient in every other circumstance. Next, in ascending order of fragility, is Commercial Lending and Credit Cards.

---

## **6. What's Not Included**

There are several directions we could take this from here. In no particular order are some ideas I had for this project, but decided not to implement to keep the scope focused

- Export to Excel.
- Feed table into Python Pandas, a library commonly used for advanced data analysis.
- Visualize with a Heatmap or Bar Chart

Additionally, I was aware that Basel III regulation is complex, and I would be best served by wrapping my head around the basics and making a strong project on those. Other aspects I did not include are the Capital Conservation Buffer, Countercyclical Capital Buffer, SIFI/GSIB Buffer, and other additional requirements that large institutions would have to take into account.